In [1]:
import pandas as pd
import numpy as np
#import plotly.express as px
#import plotly.graph_objects as go

In [56]:
data = pd.read_csv('Data/properati.csv')
data.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [62]:
data.place_name.isnull()

mask = data.place_name == 'San Miguel'
data[mask].shape[0], data[mask].geonames_id.isnull().sum()

(752, 752)

In [3]:
round((data.isnull().sum()/data.shape[0]),2)

Unnamed: 0                    0.00
operation                     0.00
property_type                 0.00
place_name                    0.00
place_with_parent_names       0.00
country_name                  0.00
state_name                    0.00
geonames_id                   0.15
lat-lon                       0.43
lat                           0.43
lon                           0.43
price                         0.17
currency                      0.17
price_aprox_local_currency    0.17
price_aprox_usd               0.17
surface_total_in_m2           0.32
surface_covered_in_m2         0.16
price_usd_per_m2              0.43
price_per_m2                  0.28
floor                         0.93
rooms                         0.61
expenses                      0.88
properati_url                 0.00
description                   0.00
title                         0.00
image_thumbnail               0.03
dtype: float64

## Geolocalización.

Hay varios campos que intervienen en la **geolocalización**. Los más útiles son **lan** y **lon** porque nos dan la ubicación exacta de la propiedad. El problema es que tienen un porcentaje alto de nulos, por lo que el objetivo va a ser tratar de inducir estos valores. 

Los campos que vamos a evaluar para definir una estrategia de imputación, serán: 
* **lat-lon:** concatenación de latitud y longitud.
* **lat:** latitud.
* **lon:** longitud.
* **place_name:** localidad o barrio.
* **place_with_parent_names:** concatenación de ubicaciones.
* **country_name:** país.
* **state_name:** provincia o partido.
* **geonames_id:** identificador de ubicación.

### Campos de Coordenadas: lat, lon y lat-lon.
Tenemos tres campos de coordenadas: **lat, lon** y **lat-lon**. Veamos qué información tienen.

In [4]:
data[['lat','lon','lat-lon']]

,lat,lon,lat-lon
0,-34.661824,-58.508839,"-34.6618237,-58.5088387"
1,-34.903883,-57.964330,"-34.9038831,-57.9643295"
2,-34.652262,-58.522982,"-34.6522615,-58.5229825"
3,-34.647797,-58.516424,"-34.6477969,-58.5164244"
4,-38.002626,-57.549447,"-38.0026256,-57.5494468"
...,...,...,...
121215,NaN,NaN,NaN
121216,NaN,NaN,NaN
121217,-34.570639,-58.475596,"-34.5706388726,-58.4755963355"
121218,NaN,NaN,NaN


In [5]:
# Cantidad de nulos
data[['lat','lon','lat-lon']].isnull().sum()

lat        51550
lon        51550
lat-lon    51550
dtype: int64

In [6]:
# Porcentaje de nulos
round(data[['lat','lon','lat-lon']].isnull().sum() / data.shape[0],2)

lat        0.43
lon        0.43
lat-lon    0.43
dtype: float64

A simple vista, pareciera ser que **lat-lon** tiene la misma información que **lat** y **lon**, con un decimal adicional.

Para comprobar si esto efectivamente es así, separaremos **lat-lon** en dos columnas y luego compararemos a cada una contra **lat** y **lon**, respectivamente.

In [7]:
# 1. Separar lat-lon en lat y lon.
lati = data['lat-lon'].apply(lambda x: x if x is np.NaN else float(x.split(",")[0]))
longi = data['lat-lon'].apply(lambda x: x if x is np.NaN else float(x.split(",")[1]))

# 2. Comparar contra lan para ver si son iguales.
lati_notnulls = lati.notnull()
lat_notnulls = data['lat'].notnull()
print('Cantidad no nulos lati:', lati_notnulls.sum())
print('Cantidad no nulos lat:', lat_notnulls.sum())
mask_noiguales_latitud = lati_notnulls != lat_notnulls
data.loc[mask_noiguales_latitud, :]

# 3. Comparar contra lon para ver si son iguales.
longi_notnulls = longi.notnull()
lon_notnulls = data['lon'].notnull()
print('Cantidad no nulos longi:', longi_notnulls.sum())
print('Cantidad no nulos lon:', lon_notnulls.sum())
mask_noiguales_longitud = longi_notnulls != lon_notnulls
data.loc[mask_noiguales_longitud, :]

# 4. Confirmación visual.
data['lat-lon_longitud'] = longi
data['lat-lon_latitud'] = lati

data[['lat-lon', 'lat', 'lat-lon_latitud','lat-lon_longitud', 'lon']]

Cantidad no nulos lati: 69670
Cantidad no nulos lat: 69670
Cantidad no nulos longi: 69670
Cantidad no nulos lon: 69670


,lat-lon,lat,lat-lon_latitud,lat-lon_longitud,lon
0,"-34.6618237,-58.5088387",-34.661824,-34.661824,-58.508839,-58.508839
1,"-34.9038831,-57.9643295",-34.903883,-34.903883,-57.964329,-57.964330
2,"-34.6522615,-58.5229825",-34.652262,-34.652262,-58.522982,-58.522982
3,"-34.6477969,-58.5164244",-34.647797,-34.647797,-58.516424,-58.516424
4,"-38.0026256,-57.5494468",-38.002626,-38.002626,-57.549447,-57.549447
...,...,...,...,...,...
121215,NaN,NaN,NaN,NaN,NaN
121216,NaN,NaN,NaN,NaN,NaN
121217,"-34.5706388726,-58.4755963355",-34.570639,-34.570639,-58.475596,-58.475596
121218,NaN,NaN,NaN,NaN,NaN


Podemos concluir que el campo **lat-lon** tiene la misma información que **lat** y **lon** separadamente. 

Por una cuestión de practicidad es preferible tener estos campos por separado, y por lo tanto se puede prescindir de esta columna.

### Columnas descriptivas de ubicación: place_name, place_with_parent_names, country_name y state_name.

In [8]:
data[['place_name', 'place_with_parent_names', 'country_name', 'state_name']]

,place_name,place_with_parent_names,country_name,state_name
0,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal
1,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur
2,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal
3,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal
4,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica
...,...,...,...,...
121215,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal
121216,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte
121217,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal
121218,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica


In [9]:
data[['place_name', 'place_with_parent_names', 'country_name', 'state_name']].isnull().sum()

place_name                 23
place_with_parent_names     0
country_name                0
state_name                  0
dtype: int64

En estas columnas aparentemente no hay mayores problemas de nulos. Sólo **place_name** tiene 23 elementos a los que les falta valor. 

Revisemos un poco más estos datos.

In [10]:
mask_place_name_nulos = data.place_name.isnull()
mask_place_name_no_nulos = data.place_name.notnull()
print('Registros no nulos:', data.loc[mask_place_name_no_nulos,'place_name'].shape[0])
print('Registros nulos:', data.loc[mask_place_name_nulos,'place_name'].shape[0])
data.loc[mask_place_name_nulos,'place_name']

Registros no nulos: 121197
Registros nulos: 23


6489     NaN
10201    NaN
11451    NaN
14839    NaN
18622    NaN
21922    NaN
23664    NaN
24722    NaN
38856    NaN
45970    NaN
46642    NaN
53130    NaN
55306    NaN
57703    NaN
57759    NaN
57764    NaN
57793    NaN
58004    NaN
58037    NaN
59069    NaN
62411    NaN
62413    NaN
63849    NaN
Name: place_name, dtype: object

In [11]:
data.loc[mask_place_name_nulos, ['place_name', 'place_with_parent_names', 'country_name', 'state_name']]

,place_name,place_with_parent_names,country_name,state_name
6489,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
10201,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
11451,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
14839,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
18622,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
21922,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
23664,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
24722,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
38856,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte
45970,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte


Al evaluar los datos se ve que ese dato está faltando y no es posible deducirlo. Como son sólo 23 registros, podríamos eliminarlos de la serie.

Nos fijamos cómo está el resto de los campos cuando estos valores son nulos, para estar seguros de no estar tirando información valiosa:

In [12]:
data.loc[mask_place_name_nulos, ['price','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2']]
# Ver si se puede deducir place_name con la descripcion

,price,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2
6489,650000.0,11468925.0,650000.0,2708.333333,2708.333333
10201,535000.0,9439807.5,535000.0,NaN,1783.333333
11451,550000.0,9704475.0,550000.0,253.456221,2037.037037
14839,480000.0,8469360.0,480000.0,2000.000000,2400.000000
18622,1297000.0,22884916.5,1297000.0,3814.705882,3814.705882
21922,440000.0,7763580.0,440000.0,NaN,2000.000000
23664,480000.0,8469360.0,480000.0,1632.653061,1632.653061
24722,530000.0,9351585.0,530000.0,1677.215190,1962.962963
38856,1350000.0,23820075.0,1350000.0,NaN,2454.545455
45970,95000.0,1676227.5,95000.0,1727.272727,3518.518519


Como las columnas de precio tienen información, decidimos que es mejor no eliminarlos.

Al revisar los datos que contiene **place_with_parent_names**, podemos ver que tiene datos que las otras tres columnas juntas no poseen.

In [13]:
data.place_with_parent_names.head(20)

0                 |Argentina|Capital Federal|Mataderos|
1           |Argentina|Bs.As. G.B.A. Zona Sur|La Plata|
2                 |Argentina|Capital Federal|Mataderos|
3                   |Argentina|Capital Federal|Liniers|
4     |Argentina|Buenos Aires Costa Atlántica|Mar de...
5                   |Argentina|Entre Ríos|Gualeguaychú|
6     |Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...
7                  |Argentina|Capital Federal|Belgrano|
8                  |Argentina|Capital Federal|Belgrano|
9                          |Argentina|Santa Fe|Rosario|
10                          |Argentina|Córdoba|Córdoba|
11      |Argentina|Bs.As. G.B.A. Zona Norte|San Miguel|
12    |Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...
13     |Argentina|Capital Federal|Palermo|Palermo Soho|
14     |Argentina|Capital Federal|Palermo|Palermo Soho|
15    |Argentina|Bs.As. G.B.A. Zona Norte|San Miguel...
16                |Argentina|Capital Federal|Mataderos|
17    |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|

Vamos a guardar estos datos en columnas separadas para futuros análisis.

In [14]:
serie_place_with_parent_names = data.place_with_parent_names

def limpiar_serie(serie):
    parents_name_lista = []
    for i in serie:
        parent_name = i.split("|")

        #se elimina el primer elemento si está vacío
        if (parent_name[0] != ''):
            print("Elemento encontrado:", parent_name[0])
        else:
            del parent_name[0]

        #se elimina el último elemento si está vacío    
        if (parent_name[-1] != ''):
            print("Elemento encontrado:", parent_name[-1])
        else:
            del parent_name[-1]

        #Se imprime la lista limpia para chequear que no hayan quedado elementos vacíos
        #print(parent_name)
        parents_name_lista.append(parent_name)
        
    return parents_name_lista

# Con la función definida obtenemos una lista con los elementos sin blancos
parents_name_list = limpiar_serie(serie_place_with_parent_names)

In [15]:
#se crea una cantidad de campos en base al valor máximo de len() 
#se recorren los elementos y, si no están nulos, se los va agregando a los campos 
longitud_maxima = 0

for pn in parents_name_list:
    if (len(pn) >= longitud_maxima):
        longitud_maxima = len(pn)
         
print("La cantidad máxima de elementos es:",longitud_maxima)

La cantidad máxima de elementos es: 5


In [16]:
# Vamos a crear las columnas necesarias y reutilizar el resto, si son iguales
serie_parents_name = pd.Series(parents_name_list)

pais_argentina = serie_parents_name.apply(lambda x: True if x[0] == 'Argentina' else False)
pais_argentina.value_counts()

True    121220
dtype: int64

In [17]:
provincia = serie_parents_name.apply(lambda x: x[1])
provincia.value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

### Columna geonames_id

In [18]:
pd.set_option('display.max_rows', None)

#Evaluamos la información en ambas columnas
data_grouped = data[['place_name','geonames_id']].groupby(['place_name','geonames_id']).sum().sort_values(by='geonames_id')
data_grouped

,
place_name,geonames_id
Zelaya,3427208.0
Zárate,3427212.0
Wilde,3427326.0
Virreyes,3427354.0
Villa Soldati,3427366.0
Villa Sarmiento,3427367.0
Villars,3427375.0
Villa Rosa,3427377.0
Villa Riachuelo,3427379.0


In [19]:
pd.reset_option('display.max_rows')

mask_geo_place_null = data.geonames_id.isnull()
df_geoname_null = data.loc[mask_geo_place_null,:]

mask_geo_place_not_null = data.geonames_id.notnull()
df_geoname_not_null = data.loc[mask_geo_place_not_null,:]

print("Cantidad de registros con geonames_id completo:", df_geoname_not_null.geonames_id.shape[0])
print("Cantidad de registros con geonames_id nulo:", df_geoname_null.geonames_id.shape[0])
print("Cantidad de registros sumados:", df_geoname_not_null.geonames_id.shape[0] + df_geoname_null.geonames_id.shape[0])
print("Cantidad de registros totales:", data.geonames_id.shape[0])

df_geoname_null.fillna(0).groupby('place_name')['geonames_id'].count().sort_values(ascending=False)

Cantidad de registros con geonames_id completo: 102503
Cantidad de registros con geonames_id nulo: 18717
Cantidad de registros sumados: 121220
Cantidad de registros totales: 121220


place_name
Nordelta                 3315
Lomas de Zamora          1253
Lanús                     971
Tigre                     965
San Miguel                752
                         ... 
Golf Club Argentino         1
Villa Amancay               1
Villa Argüello              1
Faro Norte                  1
coordenadas 34.255511       1
Name: geonames_id, Length: 467, dtype: int64

In [49]:
#Validación visual. 

#Punta Iglesia debería tener 6 geonames_id en NaN
mask_punta_iglesia = data.place_name == 'Punta Iglesia'
data.loc[mask_punta_iglesia,['place_name', 'state_name', 'geonames_id']]

,place_name,state_name,geonames_id
61499,Punta Iglesia,Buenos Aires Costa Atlántica,NaN
74403,Punta Iglesia,Buenos Aires Costa Atlántica,NaN
93480,Punta Iglesia,Buenos Aires Costa Atlántica,NaN
93939,Punta Iglesia,Buenos Aires Costa Atlántica,NaN
100043,Punta Iglesia,Buenos Aires Costa Atlántica,NaN
121139,Punta Iglesia,Buenos Aires Costa Atlántica,NaN


In [50]:
data.place_name.isnull().sum()

23

In [21]:
#Faro Norte debería tener 1 geonames_id en NaN
mask_faro_norte = data.place_name == 'Faro Norte'
data.loc[mask_faro_norte,['place_name', 'state_name', 'geonames_id']]

,place_name,state_name,geonames_id
57384,Faro Norte,Buenos Aires Costa Atlántica,NaN


In [22]:
print("Existen", data.geonames_id.isnull().sum(), "valores nulos de la columna geonames_id")
print("--------------------------------------------------------------")
data_agrupada_st_nm = data.groupby('state_name')['geonames_id'].count().sort_values(ascending=False)
print("Registros agrupados de geonames_id por state_name:", data_agrupada_st_nm.shape[0])
print("Registros únicos de geonames_id por state_name:", data_agrupada_st_nm.unique().shape[0])
print("--------------------------------------------------------------")
data_agrupada_pl_nm = data.groupby('place_name')['geonames_id'].count().sort_values(ascending=False)
print("Registros agrupados de geonames_id por place_name:", data_agrupada_pl_nm.shape[0])
print("Registros únicos de geonames_id por place_name:", data_agrupada_pl_nm.unique().shape[0])

Existen 18717 valores nulos de la columna geonames_id
--------------------------------------------------------------
Registros agrupados de geonames_id por state_name: 28
Registros únicos de geonames_id por state_name: 27
--------------------------------------------------------------
Registros agrupados de geonames_id por place_name: 1060
Registros únicos de geonames_id por place_name: 206


Para completar los valores nulos, vamos a hacerlo por **place_name** o **state_name**. Agrupando por uno de estos campos, donde haya algún **geonames_id** nulo lo completaremos con el valor del que esté completo que tenga el mismo **place_name** o **state_name** respectivamente.

Para esto, vamos a:

1. Agrupar los datos por ambos criterios, utilizando uno a la vez: **place_name** o **state_name**.
2. Dentro de esa agrupación, buscar nulos del campo **geonames_id**.
3. Si son todos los mismos números, impactar el campo vacío con el mismo valor que tiene el resto de los campos. Si los números son distintos, ver si es coherente imputar la moda.

In [23]:
df = data.groupby('place_name')
mask_geonames_nulos = data.geonames_id.isnull()
serie_pln_geonames_nulos = data.loc[mask_geonames_nulos,['place_name']]
serie_stn_geonames_nulos = data.loc[mask_geonames_nulos,['state_name']]
print("Los state_name que tienen geonames_id vacíos son: \n", serie_stn_geonames_nulos.state_name.unique())
print("--------------------------------------------------------------")
print("Los place_name que tienen geonames_id vacíos son: \n", serie_pln_geonames_nulos.place_name.unique())

Los state_name que tienen geonames_id vacíos son: 
 ['Bs.As. G.B.A. Zona Norte' 'Capital Federal' 'Bs.As. G.B.A. Zona Sur'
 'Bs.As. G.B.A. Zona Oeste' 'Buenos Aires Costa Atlántica'
 'Buenos Aires Interior' 'Tucumán' 'Chubut' 'Córdoba' 'Santa Fe'
 'San Luis' 'San Juan' 'Entre Ríos' 'Neuquén' 'Catamarca' 'Río Negro'
 'Mendoza' 'Misiones' 'Jujuy']
--------------------------------------------------------------
Los place_name que tienen geonames_id vacíos son: 
 ['San Miguel' 'Barrio El Golf' 'Pilar' 'Barrio San Gabriel'
 'Estancias del Pilar' 'Las Cañitas' 'Fincas de Iraola' 'San Martín'
 'Nordelta' 'Boulogne Sur Mer' 'Lanús Este' 'Villa Maipu'
 'Country Maschwitz Privado' 'Las Golondrinas' 'Palermo Hollywood'
 'José C Paz' 'Barrio Las Glorietas' 'Barrio Cerrado Fincas de Maschwitz'
 'Haedo' 'Lisandro Olmos' 'Parque Centenario' 'Vicente López' 'La Perla'
 'Tigre' 'Palermo Chico' 'Barrio Los Lagos' 'Martín Coronado'
 'Mapuche Country Club' 'Berisso' 'Villa de Mayo' 'Caisamar'
 'Manuel Albe

In [54]:
mask_geo_place_null = data.geonames_id.isnull()
df_geoname_null = data.loc[mask_geo_place_null,:]

mask_geo_place_not_null = data.geonames_id.notnull()
df_geoname_not_null = data.loc[mask_geo_place_not_null,:]

df_geoname_null.sum()
#df_geoname_not_null

Unnamed: 0                                                           1059510887
operation                     sellsellsellsellsellsellsellsellsellsellsellse...
property_type                 houseapartmentapartmenthousehouseapartmenthous...
place_with_parent_names       |Argentina|Bs.As. G.B.A. Zona Norte|San Miguel...
country_name                  ArgentinaArgentinaArgentinaArgentinaArgentinaA...
state_name                    Bs.As. G.B.A. Zona NorteBs.As. G.B.A. Zona Nor...
geonames_id                                                                   0
lat                                                                     -365383
lon                                                                     -617380
price                                                               6.82746e+09
price_aprox_local_currency                                          8.52956e+10
price_aprox_usd                                                     4.83412e+09
surface_total_in_m2                     

In [48]:
df_geoname_null.isnull().sum()

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                       23
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   18717
lat-lon                        8185
lat                            8185
lon                            8185
price                          1765
currency                       1765
price_aprox_local_currency     1765
price_aprox_usd                1765
surface_total_in_m2            5499
surface_covered_in_m2          2053
price_usd_per_m2               7021
price_per_m2                   3567
floor                         17911
rooms                         12166
expenses                      16407
properati_url                     0
description                       0
title                             0
image_thumbnail                 395
lat-lon_longitud               8185
lat-lon_latitud             

In [25]:
datos_agrupados = data[data.geonames_id.notnull()].groupby("place_name")["geonames_id"].value_counts()
datos_agrupados.loc['Abasto'].index[0]

3436521.0

In [64]:
contador = 0

for index,row in data[data.geonames_id.isnull()].iterrows():
    if np.isnan(row.geonames_id):
        try:
            row.geonames_id = datos_agrupados.loc[row.place_name].index[0]
            #print(row.geonames_id)
        except:
            contador+=1

print(contador)            
#data[data.geonames_id.notnull()].groupby("place_name")["geonames_id"].value_counts()
#data.groupby("place_name")["geonames_id"].value_counts()

13703


In [44]:
df_geoname_null.isnull().sum()

place_name     23
geonames_id    23
dtype: int64

In [41]:
mask = data.place_name == "Macrocentro"
df_geoname_null.loc[mask,['place_name','geonames_id']] 

,place_name,geonames_id


In [42]:
mask_geo_place_null = data.geonames_id.isnull() & data.place_name.isnull()
df_geoname_null = data.loc[mask_geo_place_null,['place_name','geonames_id']]
df_geoname_null.isnull().sum()

,place_name,geonames_id
6489,NaN,NaN
10201,NaN,NaN
11451,NaN,NaN
14839,NaN,NaN
18622,NaN,NaN
21922,NaN,NaN
23664,NaN,NaN
24722,NaN,NaN
38856,NaN,NaN
45970,NaN,NaN


In [28]:
data.place_name.value_counts()

Córdoba           9254
Rosario           8504
Mar del Plata     6534
Tigre             3324
Nordelta          3315
                  ... 
José Hernández       1
Lima                 1
Ataliva              1
Álvarez              1
El Chaltén           1
Name: place_name, Length: 1060, dtype: int64

### Correspondencia entre las coordenadas y su ubicación.
A continuación vamos a verificar si los puntos están correctamente ubicados. Es decir, si se corresponden con los barrios/localidades que dice el dataset que tiene. 

Hacemos esto para verificar si la información es fiable y se la puede utilizar para complementar otros análisis.

In [29]:
df = data
df['text'] = df['place_name']

fig = go.Figure(data=go.Scattergeo(
        lon = df['lon'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        ))

fig.update_layout(
        title = 'Propiedades',
        geo_scope="south america",
    )
#fig.show()

NameError: name 'go' is not defined